> A estratégia consiste em selecionar os business que aparecem em eval_set, selecionar as 5 melhores reviews dele e as 5 piores reviews (com menor nota), gerar embeddings dessas reviews e calcular a média.

> Depois, fazemos o mesmo para os users que aparecem em eval_set: pegamos as 5 melhores, 5 piores, tira a média dos embeddings e comparamos com as médias dos business.

> A lógica é: se os embeddings de melhor avaliação for maior, recomenda para a pessoa, se o de pior avaliação for maior, não recomenda.

> Para ordenar os business, verificamos qual é maior:
- se for o embedding de melhor avaliação, ordenamos de forma que quanto maior o valor dele, mais à frente fica.
- se for o embedding de pior avaliação, ordenamos de forma que quanto maior o valor dele, mais atrás ele fica.

In [2]:
import pandas as pd

# Carregando os dados

In [3]:
# carregando os dados do eval_set
eval_set = pd.read_csv('../data/evaluation/eval_users.csv')

In [4]:
# carregando o dataset de reviews
reviews = pd.read_parquet('../data/DatasetsLimpos/yelp_academic_dataset_review.parquet')

# Filtrando

### Filtrando apenas os business que aparecem no eval_set

In [5]:
## carregando eval_df
# verificando os business que aparecem em eval
business_ = eval_set['gt_reclist'].apply(lambda x: x.replace("[", "").replace("]", "").split(',')).explode().unique()

In [6]:
business_ = pd.Series(business_).apply(lambda x: x.replace("'", "").replace(" ", "")).unique()

In [7]:
# filtrando apenas os business que aparecem em eval
reviews = reviews[reviews['business_id'].isin(business_)]

In [8]:
reviews.shape

(1935414, 7)

In [9]:
# selecionando apenas as colunas que serão utilizadas
reviews = reviews[['business_id', 'user_id','text', 'stars']]

In [14]:
# agrupando os business
reviews = reviews.groupby('business_id').agg({'text': lambda x: [i for i in x], 'stars': lambda x: [i for i in x]})

In [26]:
import numpy as np

In [30]:
# criando uma função que recebe uma lista e retorna os índices dos 5 maiores e 5 menores valores (ou 2)
def get_best_worst_indices(lista):
    # Converte a lista para um array NumPy
    arr = np.array(lista)

    if arr.shape[0] < 10:
        # Obtém os índices dos N maiores valores usando argsort
        indices_n_maiores = np.argsort(-arr)[:2]
        indices_n_menores = np.argsort(arr)[:2]
    else:
        # Obtém os índices dos N maiores valores usando argsort
        indices_n_maiores = np.argsort(-arr)[:5]
        indices_n_menores = np.argsort(arr)[:5]
    return indices_n_maiores.tolist(), indices_n_menores.tolist()

In [ ]:
# aplicando na coluna stars
reviews['stars'] = reviews.stars.apply(get_best_worst_indices)

In [ ]:
# filtrando a coluna texto para pegar apenas as reviews que estão nos indices de stars
reviews['text'] = reviews.apply(lambda x: [x['text'][i] for i in x['stars'][0] + x['stars'][1]], axis=1)

### Gerando os embeddings dos business

In [41]:
from BertEmbedding import get_bert_embedding

In [ ]:
# aplicando a funcao no text
reviews['embs'] = reviews['text'].apply(lambda x: get_bert_embedding(x)[0])

Time to get embedding:  0.21450424194335938
Time to get embedding:  0.21860361099243164
Time to get embedding:  0.22352361679077148
Time to get embedding:  0.21901273727416992
Time to get embedding:  0.2145688533782959
Time to get embedding:  0.09015679359436035
Time to get embedding:  0.22377824783325195
Time to get embedding:  0.21724557876586914
Time to get embedding:  0.21671342849731445
Time to get embedding:  0.22028112411499023
Time to get embedding:  0.2231311798095703
Time to get embedding:  0.21578001976013184
Time to get embedding:  0.21561574935913086
Time to get embedding:  0.09072327613830566
Time to get embedding:  0.22366714477539062
Time to get embedding:  0.21712422370910645
Time to get embedding:  0.2125251293182373
Time to get embedding:  0.22505712509155273
Time to get embedding:  0.2139110565185547
Time to get embedding:  0.2199089527130127
Time to get embedding:  0.21856021881103516
Time to get embedding:  0.22296881675720215
Time to get embedding:  0.21563339233

In [ ]:
# separando os embeddings em colunas: melhor avaliados [:5] e piores avaliados [5:]
reviews['embs_best'] = reviews['embs'].apply(lambda x: x[:5] if x.shape[0] == 10 else x[:2])
reviews['embs_worst'] = reviews['embs'].apply(lambda x: x[5:] if x.shape[0] == 10 else x[2:])

### Filtrando os users que aparecem em eval_set